In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pyperclip as pc
import pandas as pd
import re
import time

In [2]:
# url cleansing function to get coordinate
def getCoordinate(url):
    x = re.search("(?<=@).*(?=,)", url)
    return x.group()

# get all searching results function
def getResults(results, name, address, coordinate):
    for x in range(len(results)):
        result = driver.find_element_by_xpath('//div[@class="section-result"]['+ str(x + 1) + "]")
        result.click()
        time.sleep(2)
        
        try: # check is there full address
            address_txt = driver.find_element_by_xpath('//img[@alt="Salin alamat"]')
        except:
            getAddress()
        else:
            name_txt = driver.find_element_by_xpath('//h1/span[1]')
            address_txt = driver.find_element_by_xpath('//img[@alt="Salin alamat"]')
            back_btn = driver.find_element_by_xpath('//span[contains(text(),"Kembali ke hasil")]')

            zoomIn()
            address_txt.click()

            # save to list
            name.append(name_txt.text)
            address.append(pc.paste())
            coordinate.append(getCoordinate(driver.current_url))

            back_btn.click()
            time.sleep(2)
    return name, address, coordinate

# get address function when it can't copying to clipboard
def getAddress(name, address, coordinate):
    try:
        place_info = driver.find_elements_by_xpath('//h2/span')
    except:
        driver.back()
    else:
        name_txt = driver.find_element_by_xpath('//h1/span[1]')

        name.append(name_txt.text)
        address.append((', ').join(list(map(lambda i : i.text, place_info))))
        coordinate.append(getCoordinate(driver.current_url))

        driver.back()
        driver.back()
        time.sleep(2)
    return name, address, coordinate

# zoom-in function
def zoomIn():
    zoom_btn = driver.find_element_by_xpath('//button[@id="widget-zoom-in"]')
    while (zoom_btn.get_attribute('disabled') != 'true'):
        zoom_btn.click()

In [3]:
# scrapping function
def scrappingMaps(areas, keyword):
    name = []
    address = []
    coordinate = []
    
    # open webdriver
    driver = webdriver.Firefox()
    driver.get('http://www.google.com/maps')

    for area in areas:
        search_box = driver.find_element_by_id('searchboxinput')
        submit_button = driver.find_element_by_id('searchbox-searchbutton')

        search_box.clear()
        search_box.send_keys(area)
        search_box.send_keys(Keys.ENTER)
        time.sleep(5)
        search_box.clear()
        search_box.send_keys(keyword)
        search_box.send_keys(Keys.ENTER)
        time.sleep(5)
        
        # get search results
        results = driver.find_elements_by_xpath('//div[@class="section-result"]')

        if len(results) != 0:
            next_btn = driver.find_element_by_xpath('//button[@aria-label="Halaman berikutnya"]')
            
            # loop while next button undisable
            while (next_btn.get_attribute('disabled') != True):
                results = driver.find_elements_by_xpath('//div[@class="section-result"]')
                getResults(results, name, address, coordinate)

                next_btn = driver.find_element_by_xpath('//button[@aria-label="Halaman berikutnya"]')
                next_btn.click()
                time.sleep(3)

        else: # if no result
            try: # check is there full address
                address_txt = driver.find_element_by_xpath('//img[@alt="Salin alamat"]')
            except:
                getAddress(name, address, coordinate)
            else:
                name_txt = driver.find_element_by_xpath('//h1/span[1]')
                address_txt = driver.find_element_by_xpath('//img[@alt="Salin alamat"]')

                zoomIn()
                address_txt.click()

                name.append(name_txt.text)
                address.append(pc.paste())
                coordinate.append(getCoordinate(driver.current_url))

                driver.back()
                driver.back()
                time.sleep(2)
    
    # close webdriver
    driver.close()
    
    # storing scrapped data
    data = {'Name' : name, 'Address': address, 'Coordinate':coordinate}
    return data

In [5]:
# main
areas = ['Tambun','Gunung Sindur']
keyword = 'dahlia loka'

scrapped = scrappingMaps(areas, keyword)
scrapped

{'Name': ['Dahlia Loka', 'Jl. Graha Raya Dahlia Loka'],
 'Address': ['Blok RB 04, Jl. Graha Raya Dahlia Loka No.07, Pd. Jagung Tim., Kec. Serpong Utara, Kota Tangerang Selatan, Banten 15326',
  'Pd. Jagung Tim., Kec. Serpong Utara, Kota Tangerang Selatan, Banten 15326'],
 'Coordinate': ['-6.2447905,106.6695283', '-6.2448168,106.66655']}

In [6]:
# print dataframe
df = pd.DataFrame(scrapped)
df

,Name,Address,Coordinate
0,Dahlia Loka,"Blok RB 04, Jl. Graha Raya Dahlia Loka No.07, ...","-6.2447905,106.6695283"
1,Jl. Graha Raya Dahlia Loka,"Pd. Jagung Tim., Kec. Serpong Utara, Kota Tang...","-6.2448168,106.66655"


In [9]:
# save to csv
# df.to_csv(keyword +'_Maps-Scrapped.csv', index = False, header = True, sep = ';')